In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import  LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

In [2]:
# 밑에 '''''' 는 확인해보시고 쓸지안쓸지? 
# 상관분석도 어떤걸로 쓸건지? 
# 기존 df로 선택해서 갈건지 아니면 차원축소한 버전으로 갈건지?

# 데이터 불러오기

In [3]:
df = pd.read_csv('data/apt161718.csv', low_memory=False, dtype={'status': 'category'})

FileNotFoundError: [Errno 2] No such file or directory: 'data/apt161718.csv'

# 데이터 EDA 및 전처리 1

# 분석에 맞는 데이터 필터링 & 컬럼 정리

In [ ]:
df.head()
# 문자, 숫자 섞여 있는 데이터

In [ ]:
df.shape
# 4,822,002 개의 레코드 데이터, 21개 컬럼

In [ ]:
# 컬럼 : 시군구, 번지, 본번, 부번, 단지명, 전월세구분, 전용면적(㎡), 계약년월, 계약일, 보증금(만원), 월세(만원), 층, 건축년도, 도로명, 계약기간, 계약구분,
#         갱신요구권 사용, 종전계약 보증금 (만원), 종전계약 월세 (만원), 금리, 부동산소비심리


# 컬럼 중 시군구, 번지, 본번, 부번, 도로명은 서로 성격이 겹치는 컬럼이기에 하나로 대표할 수 있는 시군구 컬럼만 남김
# 컬럼 중 계약년월, 계약일 중 계약일 컬럼만 남김
# 알아내고자 하는 것이 보증금이기 때문에 계약기간(유효한 값이 거의 없기에 그냥 삭제하기로함), 계약구분, 개신요구권사용, 종전계약보증금(만원), 종전계약 월세(만원)은 삭제하여 활용하기로 함 
# 이중에서도 전세에 관한 것을 알기 위해 전월세구분에서 전세로 필터링해 활용함


# 단어 의미 확인하기
# 시군구 : 행정 구역인 시와 군, 구를 아울러 이르는 말
# 단지명 : 주택, 공장, 작물 재배지 따위가 집단을 이루고 있는 일정 구역의 이름
# 전세 : 부동산의 소유주에게 일정한 돈을 맡기고 집이나 방을 빌려 쓰다가, 내놓을 때 그 돈의 전액을 돌려받는 제도 
# 전용면적 : 아파트 따위의 공동 주택에서, 출입구나 엘리베이터, 계단 등의 공유 면적을 제외한 바닥 넓이
# 계약년월 : 계약을 진행한 연도와 월
# 보증금 : 입찰 또는 계약을 맺을 때 계약 이행의 담보로 내는 금전
# 층 : 건물의 층 
# 건축년도 : 건물 완공 년
# 금리 : 금융 기관에서 빌려준 돈이나 예금 따위에 대한 이율
# 부동산소비심리 : 부동산에서 반응하는 소비자들의 심리 수준, 형태 변화 및 인지 수준을 파악한 뒤 지수화한 값

In [ ]:
#우리가 알고자 하는 주제는 전세이기 때문에 필터링 진행
df = df[df['전월세구분'] == '전세']


# 서울시 동별 보증금이 다를 것이라는 가설을 전제로 '동' 컬럼 추가
df['동'] = df['시군구'].str.replace('서울특별시', '')
df = df[df['전월세구분'] == '전세']
remove_gu = ['서울특별시 ', '강남구 ', '강동구 ', '강북구 ', '강서구 ', '관악구 ', '광진구 ','구로구 ', '금천구 ', '노원구 ', '도봉구 ', '동대문구 ', '동작구 ', '마포구 ', '서대문구 ', '서초구 ', '성동구 ', '성북구 ','송파구 ' , '양천구 ', '영등포구 ', '용산구 ', '은평구 ', '종로구 ', '중구 ', '중랑구 ']
df['동'] = df['시군구'].str.replace('|'.join(remove_gu), '', regex=True)


#데이터 보기 편하게 재배열
df = df[['시군구', '동', '단지명','전월세구분','전용면적(㎡)','계약년월','층', '건축년도', '금리','부동산소비심리', '보증금(만원)']]


# 보증금(만원) 컬럼이 문자로 저장되어 있어 숫자로 변환
df['보증금(만원)'] = df['보증금(만원)'].str.replace(",", "").astype(int)


# 전용면적(㎡), 보증금(만원) 컬럼명에서 ()가 들어가 삭제함
df.rename(columns={'전용면적(㎡)': '전용면적'}, inplace=True)
df.rename(columns={'보증금(만원)': '보증금'}, inplace=True)



# 각 컬럼에 대한 특징 파악하기

In [ ]:
# 1) 피처(Feature) 확인
df.columns

In [ ]:
# 2) 데이터 프레임 정보 확인
df.info()

In [ ]:
# 3) 데이터 프레임 통계값 (전체)
df.describe()

In [ ]:
# 동 : 서울의 동을 나누었을 때 332개로 나누었으며(충무로1가, 장충동2가 등 포함) 가장 많은 거래(12,727건)가 상계동에서 있었다는 것을 알 수 있음
df.동.describe()

In [ ]:
# 단지명 : 6735의 서로 다른 아파트들이 있다는 것을 알 수 있음 -> 아파트명이 많이 겹침
# 겹쳐진 아파트병을 하나로 합칠 것인가 고민 -> 동일명아파트인데도 지역에 따라 전세 가격이 1억3천, 5억 이런 식으로 같은 아파트명인데도 차이가 많이남
# 퍼포먼스가 많이 느리지만 우선은 감안하고 신뢰도 및 정화도 확보에 우선을 두고 우선은 진행하도록 함
# 하지만 이것보다 1.5배 이상 더 많아지는 경우에는 이름을 그루핑해 전처리해 퍼포먼스 올릴 필요 있음
df.단지명.describe()

In [ ]:
df['단지명'].value_counts() 
# 아파트에 대한 전처리는 이후 보증금에 대한 이상치 확인 후 진행

In [ ]:
# 전용면적 : 거래된 아파트 중 가장 작은 전용면적은 10.11(㎡), 가장 넓은 전용면적은 317.36(㎡) 임
df.전용면적.describe()

In [ ]:
plt.hist(df['전용면적'], bins=10, facecolor='blue', alpha=0.5)
plt.show()
#대부분 50~100사이의 면적거래가 있었다는 것을 알 수 있음

In [ ]:
# 계약년월 : 201601~201812 까지의 자료임을 알 수 있음
df.계약년월.describe()

In [ ]:
# 층 : -2~68층까지 분포 (지하층이 있는지 확인)
df.층.describe()

In [ ]:
#지하층 있는 곳 파악 , -1층, 19곳
df.loc[df['층']==-1]

In [ ]:
#지하층 있는 곳 파악 , -2층, 6곳
df.loc[df['층']==-2]

In [ ]:
# 건축년도 : 1961년도~2022년도 신축까지 다양함
# 왜 16~18 전세 자료인데 건축년도가 22년이 나오는가?
# 장기계속공사의 경우 건설공사계약통계는 당해 연도 계약액을 기준으로 하고 있으며, 통계청 조사는 총 공사액을 기준으로 하고 있기 때문
df.건축년도.describe()

In [ ]:
# 19년, 20년, 22년 건축년도와 계약년도를 맞춤
# 16~18 년도 계약이지만 건축년도가 미래에 있음. 그러나 전세 보증금의 기준은 계약년도였기 때문에 건축년도를 맞추어 데이터의 모순성 제거
search = df[(df['건축년도'] == 2019) & (df['계약년월'] == 201707)]
print(search)

In [ ]:
# 건축년도 2019년, 계약년월 201707 에 1건
# 건축년도 2019년, 계약년월 201712 에 1건
# 건축년도 2019년, 계약년월 201801 에 89건
# 건축년도 2019년, 계약년월 201802 에 71건
# 건축년도 2019년, 계약년월 201803 에 141건
# 건축년도 2019년, 계약년월 201804 에 118건
# 건축년도 2019년, 계약년월 201805 에 97건
# 건축년도 2019년, 계약년월 201806 에 120건
# 건축년도 2019년, 계약년월 201807 에 135건
# 건축년도 2019년, 계약년월 201808 에 118건
# 건축년도 2019년, 계약년월 201809 에 116건
# 건축년도 2019년, 계약년월 201810 에 129건
# 건축년도 2019년, 계약년월 201811 에 150건
# 건축년도 2019년, 계약년월 201812 에 278건
# ...

In [ ]:
df.loc[(df['계약년월'].isin([201601, 201602, 201603, 201604, 201605, 201606, 201607, 201608, 201609, 201610, 201611, 201612])) & (df['건축년도'] == 2019), '건축년도'] = 2016
df.loc[(df['계약년월'].isin([201601, 201602, 201603, 201604, 201605, 201606, 201607, 201608, 201609, 201610, 201611, 201612])) & (df['건축년도'] == 2020), '건축년도'] = 2016
df.loc[(df['계약년월'].isin([201601, 201602, 201603, 201604, 201605, 201606, 201607, 201608, 201609, 201610, 201611, 201612])) & (df['건축년도'] == 2021), '건축년도'] = 2016
df.loc[(df['계약년월'].isin([201601, 201602, 201603, 201604, 201605, 201606, 201607, 201608, 201609, 201610, 201611, 201612])) & (df['건축년도'] == 2022), '건축년도'] = 2016

df.loc[(df['계약년월'].isin([201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708, 201709, 201710, 201711, 201712])) & (df['건축년도'] == 2019), '건축년도'] = 2017
df.loc[(df['계약년월'].isin([201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708, 201709, 201710, 201711, 201712])) & (df['건축년도'] == 2020), '건축년도'] = 2017
df.loc[(df['계약년월'].isin([201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708, 201709, 201710, 201711, 201712])) & (df['건축년도'] == 2021), '건축년도'] = 2017
df.loc[(df['계약년월'].isin([201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708, 201709, 201710, 201711, 201712])) & (df['건축년도'] == 2022), '건축년도'] = 2017

df.loc[(df['계약년월'].isin([201801, 201802, 201803, 201804, 201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812])) & (df['건축년도'] == 2019), '건축년도'] = 2018
df.loc[(df['계약년월'].isin([201801, 201802, 201803, 201804, 201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812])) & (df['건축년도'] == 2020), '건축년도'] = 2018
df.loc[(df['계약년월'].isin([201801, 201802, 201803, 201804, 201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812])) & (df['건축년도'] == 2021), '건축년도'] = 2018
df.loc[(df['계약년월'].isin([201801, 201802, 201803, 201804, 201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812])) & (df['건축년도'] == 2022), '건축년도'] = 2018

In [ ]:
# 확인
search2 = df[(df['건축년도'] == 2019) & (df['계약년월'] == 201707)]
print(search2)

In [ ]:
df

In [ ]:
# 보증금 : 제일 저렴한 것은 500만원, 비싼 것은 500000만원(50억.. ㅎ)이 나온다는 것을 알 수 있음 
df.보증금.describe()

In [ ]:
#저렴한 곳 : 500만원
df.loc[df['보증금']==500]

In [ ]:
#비싼곳 : 50억원
df.loc[df['보증금']==500000]

In [ ]:
plt.hist(df['보증금'], bins=10, facecolor='blue', alpha=0.5)
plt.show()
# 10억이하, 5억 이하 주위로 많이 분포되어 있음을 알 수 있음

In [ ]:
plt.boxplot(df['보증금'])
plt.show()

In [ ]:
# 누가봐도 10억 이상은 안될 것 같은 그래프 나옴
# 10억 이상의 데이터를 이상치로 잡고 제거 or 평균화 -> 제거

In [ ]:
# 보증금이 10억 이상인 데이터 카운트
df_10억 = df[df['보증금'] > 100000]
print(df_10억)

In [ ]:
# 보증금이 5억 이상인 데이터 카운트
df_5억 = df[df['보증금'] >= 50000]
print(df_5억)

In [ ]:
# 전체 328,695 건 중 10억 초과인 경우 : 7,716건 -> 전체의 약 2.3%

In [ ]:
# 전체 328,695 건 중 5억 이상인 경우 : 90,337건 -> 전체의 약 27.4% 

In [ ]:
# 3분위수 이상값(5억이상)을 이상치로 잡을 것인가 vs 10이상을 이상치로 잡을 것인가
# 값에 영향을 많이 받지 않도록 10억 이상의 값을 이상치로 잡아 하는 것으로?? 

# 값에 영향을 많이 받지 않도록 10억 이상의 값을 이상치로 잡아 하는 것으로??

# 값은 삭제할 것인지 아니면 평균값(또는 중간값)으로 넣을 것인지? 아니면 최대값으로 잡은 10억으로 잡을 것인지?, 비싼 것을 평균값으로 넣는것은 신뢰도가 떨어지고 최대값으로 넣으면 10억 자체가 이상치로 발견될 수 있기에 그냥 삭제하자.

In [ ]:
# 삭제진행

# 필터
df = df[df['보증금'] <= 100000]

# 이상치 데이터 삭제 후 확인 : 최대 10억, 최소 500, 중간값 3억7천, 평균값 3억 9천
df.describe()

In [ ]:
# 아파트별 가격 평균 내림차순으로 정렬
apt_price = df.groupby('단지명')['보증금'].agg('mean').sort_values(ascending=False)
print('변환전\n', apt_price[:5])

In [ ]:
df.head()

In [ ]:
savemydata = df
savemydata.head()

savemydata[['계약년월', '보증금']].to_csv('data/savemydata.csv', index=False)

In [ ]:
#가격별로 0~ 인코딩 해주기
for i, a in enumerate(list(apt_price.index)):
    df.loc[df['단지명'] == a, '단지명'] = i 
apt_price = df.groupby('단지명')['보증금'].agg('mean').sort_values(ascending=False)
print('변환후\n', apt_price[:5])

In [ ]:
# 완공연도에서 최소연도 빼서 완공연도 라벨인코딩
print('변환전\n', df['건축년도'].unique()[:5])
df['건축년도'] = df['건축년도'] - df['건축년도'].min()
print('변환후\n', df['건축년도'].unique()[:5])

In [ ]:
# 연월 증가하는 순으로 라벨 인코딩 : 계약년월
le = LabelEncoder()
df['계약년월'] = le.fit_transform(df['계약년월'])
print('변환후\n', df['계약년월'].unique()[:5])

In [ ]:
# 중간 확인
df.head()

In [ ]:
# 동 인코딩하기

#동 EDA
dong_price = df.groupby('동')['보증금'].agg('mean').sort_values(ascending=False)
dong_price[:20]

#현재 데이터에서는 가장 비싼 동은 장충동1가 (10억 필터링 전에는 용산구5가였음)

In [ ]:
# 동별 인코딩 (평균값 비싼순서대로~)
for i, d in enumerate(list(dong_price.index)):
    df.loc[df['동'] == d, '동'] = i

df

In [ ]:
# 층 수 확인 
# 최소값이 -2이므로 2를 더해서 음수를 없애고 순서형 범주 처리
# -2, -1, 1, 2와 같은 숫자와 함께 "층"이라는 레이블이 있음
# 머신러닝 모델이 층수를 이해하기 위해서는 층수를 숫자만으로 변환하는 것뿐만 아니라 레이블의 변환도 필요
print('변환전\n', df['층'].values[:5])
df['층'] = df['층'].map(lambda x: x+2)
print('변환후\n', df['층'].values[:5])

In [ ]:
# 시군구,전세,apt_counts,transformed 컬럼 삭제
df = df.drop(['시군구', '전월세구분'], axis=1)
df.head()

In [ ]:
df.shape

# 변수 살펴보기

# 상관분석
### -1~1 값, 0으로 가까울수록 관계없음, -1과 1로 갈 수록 음과 양의 관계 있음

In [ ]:
# 상관분석1: 상관 관계 행렬

df_copy = df.copy()

# 분석 보기 편하기 위해 컬럼명 변경
df_copy = df_copy[['동', '단지명', '전용면적', '계약년월', '층', '건축년도', '부동산소비심리','금리','보증금']]
df_copy.columns = ['dong', 'apt_name', 'area', 'contract_month', 'floor', 'construction_year', 'psychology','interest','deposit']


#범주화된 항목 카테고리화 하기
df_copy['dong'] = df_copy['dong'].astype('category').cat.codes
df_copy['apt_name'] = df_copy['apt_name'].astype('category').cat.codes

#상관관계 행렬
df_corr=df_copy.corr()

#히트맵
plt.figure(figsize=(10,10))
sns.set(font_scale=0.8)
sns.heatmap(df_corr, annot=True, cmap="viridis")
plt.show()

In [ ]:
# 보증금과 상관관계가 높은 순으로 출력하기
corr_order = df_copy.corr()['deposit'].abs().sort_values(ascending=False)
corr_order

In [ ]:
print(df_corr)

In [ ]:


# psychology와 interest 간의 상관관계가 높음
# psychology와 interest 중 하나를 제거 가능

In [ ]:
# 상관분석2 :  선형회귀로 확인하기
# 시각화로 상관분석할 피처 정하기1 : deposit 중심
plot_cols = ['deposit', 'area','floor','construction_year','contract_month','psychology','interest']
plot_df = df_copy.loc[:, plot_cols]

plot_df.head()

In [ ]:
# regplot으로 선형회귀선 표시하기

plt.figure(figsize=(10, 10))
for idx, col in enumerate(plot_cols[1:]):
    ax1 = plt.subplot(4, 2, idx + 1)
    sns.regplot(x=col, y=plot_cols[0], data=plot_df, ax=ax1, color='gray', line_kws={'color': 'red'})
plt.tight_layout()
plt.show()

In [ ]:
# 보증금과 비교했을 시
# 면적, 층, 계약년도은 양의 상관관계가 있음
# 건축년도, 금리, 심리 : 미약한 상관관계가 보임

In [ ]:
# 피처별로 살펴보기
# 면적(area)와 다른 변수간의 관계
plot_cols = ['area', 'deposit', 'floor','construction_year','contract_month','psychology','interest']
plt.figure(figsize=(10, 10))
for idx, col in enumerate(plot_cols[1:]):
    ax1 = plt.subplot(4, 2, idx + 1)
    sns.regplot(x=col, y=plot_cols[0], data=plot_df, ax=ax1, color='gray', line_kws={'color': 'red'})
plt.tight_layout()
plt.show()

In [ ]:
'''
# 피처별로 살펴보기
# 층(floor)와 다른 변수간의 관계
plot_cols = ['floor', 'area', 'deposit', 'construction_year', 'contract_month', 'interest', 'psychology']

plt.figure(figsize=(10, 10))
for idx, col in enumerate(plot_cols[1:]):
    ax1 = plt.subplot(4, 2, idx + 1)
    sns.regplot(x=col, y='floor', data=df, ax=ax1, color='gray', line_kws={'color': 'red'})
plt.tight_layout()
plt.show()
'''

In [ ]:
'''
# 피처별로 살펴보기
# 건축년도(construction_year)와 다른 변수간의 관계
plot_cols = ['construction_year', 'area', 'deposit', 'floor','contract_month','psychology','interest']
plt.figure(figsize=(10, 10))
for idx, col in enumerate(plot_cols[1:]):
    ax1 = plt.subplot(4, 2, idx + 1)
    sns.regplot(x=col, y=plot_cols[0], data=df, ax=ax1, color='gray', line_kws={'color': 'red'})
plt.tight_layout()
plt.show()
'''

In [ ]:
'''
# 피처별로 살펴보기
# 계약년월(contract_month)와 다른 변수간의 관계
plot_cols = ['contract_month', 'construction_year', 'area', 'deposit', 'floor','psychology','interest']
plt.figure(figsize=(10, 10))
for idx, col in enumerate(plot_cols[1:]):
    ax1 = plt.subplot(4, 2, idx + 1)
    sns.regplot(x=col, y=plot_cols[0], data=df, ax=ax1, color='gray', line_kws={'color': 'red'})
plt.tight_layout()
plt.show()
'''

In [ ]:
'''
# 피처별로 살펴보기
# 부동산소비심리(psychology)와 다른 변수간의 관계
plot_cols = ['psychology', 'contract_month', 'construction_year', 'area', 'deposit', 'floor','interest']
plt.figure(figsize=(10, 10))
for idx, col in enumerate(plot_cols[1:]):
    ax1 = plt.subplot(4, 2, idx + 1)
    sns.regplot(x=col, y=plot_cols[0], data=df, ax=ax1, color='gray', line_kws={'color': 'red'})
plt.tight_layout()
plt.show()
'''

In [ ]:
'''
# 피처별로 살펴보기
# 금리(interest)와 다른 변수간의 관계
plot_cols = ['interest','psychology', 'contract_month', 'construction_year', 'area', 'deposit', 'floor']
plt.figure(figsize=(10, 10))
for idx, col in enumerate(plot_cols[1:]):
    ax1 = plt.subplot(4, 2, idx + 1)
    sns.regplot(x=col, y=plot_cols[0], data=df, ax=ax1, color='gray', line_kws={'color': 'red'})
plt.tight_layout()
plt.show()
'''

In [ ]:
# 상관분석3 : 스피어만 상관계수로 확인해보기
# 두 변수가 선형적으로 관련되어 있을때 보는 상관분석

In [ ]:
df_copy = df.copy()
df_copy = df_copy[['동', '단지명', '전용면적', '계약년월', '층', '건축년도', '부동산소비심리','금리','보증금']]
df_copy.columns = ['dong', 'apt_name', 'area', 'contract_month', 'floor', 'construction_year', 'psychology','interest','deposit']

# 범주형 변수 순위 변수로 변환
df_copy['dong'] = df_copy['dong'].astype('category').cat.codes
df_copy['apt_name'] = df_copy['apt_name'].astype('category').cat.codes

# 스피어만 상관계수 계산
r = df_copy.corr(method='spearman')

# 스피어만 상관계수를 출력
print(r)

# 스피어만 상관계수 읽는 방법
# 1에 가까울수록 두 변수 간의 상관관계가 강하고, -1에 가까울수록 두 변수 간의 상관관계가 약함 (설사약과 변비약)
# 스피어만 상관계수 값이 0에 가까울수록 두 변수 간의 상관관계가 없음

In [ ]:
# 행렬의 다른 상관 관계는 모두 비교적 약하며, 대부분은 통계적으로 유의하지 않음
# 이는 데이터 세트의 다른 변수 쌍 사이에는 강한 관계가 없다는 것을 의미함
# 물론, area와 deposit과의 사이에는 강한 상관관계가 보이지만 area가 학습 변수가 될 예정이기 때문에 논외로 침

In [ ]:
# 히트맵으로 확인
plt.imshow(r)
plt.xticks(np.arange(len(r.columns)), r.columns, rotation='vertical')
plt.yticks(np.arange(len(r.columns)), r.columns)
plt.colorbar()
plt.show()

In [ ]:
# 산점도로 확인하기
'''
plt.figure(figsize=(10, 10))
for idx, col in enumerate(plot_cols[1:]):
  ax1 = plt.subplot(4, 2, idx + 1)
  sns.regplot(x=col, y=plot_cols[0], data=plot_df, ax=ax1, color='gray', line_kws={'color': 'red'})
  ax1.text(x=0, y=1, s='Spearman correlation: {0:.3f}'.format(plot_df[col].corr(plot_df['deposit'], method='spearman')))
plt.tight_layout()
plt.show()
'''

# 변수 간 연관분석 - 주성분분석

In [ ]:
# PCA를 수행 데이터
df_copy = df.copy()
df_copy = df_copy[['동', '단지명', '전용면적', '계약년월', '층', '건축년도', '부동산소비심리','금리','보증금']]
df_copy.columns = ['dong', 'apt_name', 'area', 'contract_month', 'floor', 'construction_year', 'psychology','interest','deposit']

# PCA를 수행
pca = PCA()
pca.fit(df_copy)

# PCA 결과 해석
print(pca.explained_variance_ratio_)

In [ ]:
# 첫 번째 주성분이 데이터의 99.97%의 분산을 설명

In [ ]:
'''
from sklearn.decomposition import PCA

# 주성분을 pcs 변수에 저장. 주성분은 데이터의 주요 변수
pcs = pca.components_

# 주성분이 설명하는 분산을 var_explained 변수에 저장(분산: 데이터의 다양성)
var_explained = pca.explained_variance_ratio_

# 분산이 90% 이상인 주성분만을 선택하여 pcs 변수에 저장
pcs = pcs[:, var_explained > 0.9]
reduced_df = pca.transform(df_copy)
'''

In [ ]:
# reduced_df

In [ ]:
 # 2번째 주성분을 사용하면 데이터의 차원이 두 개로 늘어나지만, 데이터의 성능은 향상되지 않고, 데이터의 성능이 떨어짐

# 차원축소해보기

In [ ]:
# 2차원으로 만들기
# 데이터표준화
# 데이터의 크기를 동일하게 만드는 과정, 데이터차원 축소 & 데이터 특징을 비교 용이
scaler = StandardScaler()
df_copy = scaler.fit_transform(df_copy)


# 데이터의 공분산 행렬을 계산합니다.
# 공분산행렬 : 변수 간의 상관 관계 보여줌. 공분산행렬의 값이 크면 두 변수가 강하게 관련되어 있고, 공분산행렬의 값이 작으면 두 변수가 약하게 관련
cov = np.cov(df_copy.T)

# 공분산 행렬의 고유값과 고유벡터 계산
# 고유값:  행렬이 데이터의 변수 간의 상관 관계를 나타내는 척도
# 고유벡터: 행렬이 데이터의 변수 간의 관계를 나타내는 방향
eig_vals, eig_vecs = np.linalg.eig(cov)

In [ ]:
# 분산이 큰 2개만 선택 (2차원 선택)
# 고유값의 크기가 큰 고유벡터를 선택
top_n = 2
eig_vecs = eig_vecs[:, :top_n]

new_df = df.dot(eig_vecs)

# 주성분의 설명력 계산
explained_variance = eig_vals[:top_n].sum() / cov.sum()

# 주성분의 설명력
print(f'주성분의 설명력: {explained_variance}')

# 0.65

In [ ]:
# 3차원 선택

# 고유값의 크기가 큰 고유벡터를 선택
top_n = 3
eig_vecs = eig_vecs[:, :top_n]

new_df3d = df.dot(eig_vecs)

explained_variance_3d = eig_vals[:top_n].sum() / cov.sum()

# 주성분의 설명력
print(f'주성분의 설명력: {explained_variance_3d}')

# 0.67

In [ ]:
# 4차원  선택

# 고유값의 크기가 큰 고유벡터를 선택
top_n = 4
eig_vecs = eig_vecs[:, :top_n]

new_df4d = df.dot(eig_vecs)

explained_variance_4d = eig_vals[:top_n].sum() / cov.sum()

# 주성분의 설명력
print(f'주성분의 설명력: {explained_variance_4d}')

#0.70

In [ ]:
# 5차원  선택

# 고유값의 크기가 큰 고유벡터를 선택
top_n = 5
eig_vecs = eig_vecs[:, :top_n]

new_df5d = df.dot(eig_vecs)

explained_variance_5d = eig_vals[:top_n].sum() / cov.sum()

# 주성분의 설명력
print(f'주성분의 설명력: {explained_variance_5d}')

# 0.87

In [ ]:
# 6차원  선택

# 고유값의 크기가 큰 고유벡터를 선택
top_n = 6
eig_vecs = eig_vecs[:, :top_n]

new_df6d = df.dot(eig_vecs)

explained_variance_6d = eig_vals[:top_n].sum() / cov.sum()

# 주성분의 설명력
print(f'주성분의 설명력: {explained_variance_6d}')

# 0.94

In [ ]:
# 7차원  선택

# 고유값의 크기가 큰 고유벡터를 선택
top_n = 7
eig_vecs = eig_vecs[:, :top_n]

new_df7d = df.dot(eig_vecs)

explained_variance_7d = eig_vals[:top_n].sum() / cov.sum()

# 주성분의 설명력
print(f'주성분의 설명력: {explained_variance_7d}')

# 1.03 : 과대적합 상태

In [ ]:
# 주성분은 원래 데이터의 특성을 재조합하여 만든 새로운 특성 : 컬럼이름 없어짐

# 학습 데이터 결정하기 : 확인 후 결정

In [ ]:
df.head()
# 0.9997

In [ ]:
df.to_csv('data/df.csv', index=False)

In [ ]:
new_df6d.head()
# 0.94

In [ ]:
new_df6d.to_csv('data/new_df6d.csv', index=False)

# Prophet 모델링

In [ ]:
# Prophet import
import prophet
from prophet import Prophet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

In [ ]:
# 계약년월당 평균보증금

dff = pd.read_csv('data/average.csv', low_memory=False, dtype={'status': 'category'})
yeardate = pd.read_csv('data/savemydata.csv', low_memory=False, dtype={'status': 'category'})
data2019 = pd.read_csv('data/2019average.csv', low_memory=False, dtype={'status': 'category'})
alldata = pd.read_csv('data/2016-2019avg.csv', low_memory=False, dtype={'status': 'category'})


dff = dff.rename(columns={'계약년월': 'ds'})
dff = dff.rename(columns={'평균보증금': 'y'})

yeardate = yeardate.rename(columns={'계약년월': 'ds'})
yeardate = yeardate.rename(columns={'보증금': 'y'})

print(dff)

In [ ]:

yeardate['ds'] = yeardate['ds'].astype(str).str[:4] + '-' + yeardate['ds'].astype(str).str[4:] + '-01'
yeardate
yeardate[['ds', 'y']].to_csv('data/yeardate.csv', index=False)

In [ ]:
# 훈련용, 검증용 데이터 분리
# Prophet은 내부적으로 자동적으로 차분(differencing)을 수행하여 추세 및 계절성 구성 요소를 추정하고 예측함! 
# 이러한 이유로 Prophet을 사용할 때 일반적으로 차분을 직접 적용할 필요없음.

train_size = int(0.8 * len(dff))
X_train = dff[:train_size]
X_val = dff[train_size:]

X_train.shape

In [ ]:
prophet_model = Prophet(changepoint_prior_scale=0.04, seasonality_mode='additive', seasonality_prior_scale = 8)
#prophet_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# changepoint_prior_scale = 추세에 덜 유연하게 반영하려면 default 값인 0.05 보다 작게(과소적합 문제), 유연하게 반영하려면 0.05 보다 크게 설정(과적합 문제)
# seasonality_mode = 'additive' : 시계열 데이터가 진폭이 일정할 때 사용, 'multiplicative' : 시계열 데이터의 진폭이 점점 증가하거나 감소할 때 사용
# seasonality_prior_scale = 계절의 유연성을 제어하는 parameter 값이 크면 계절성의 영향이 커지고 값이 작아지면 계절성의 영향이 줄어듦. 합리적인 조정 범위는 [0.01,10] 사이
"""
prophet_model.add_regressor('동')
prophet_model.add_regressor('단지명')
prophet_model.add_regressor('전용면적')
prophet_model.add_regressor('건축년도')
prophet_model.add_regressor('금리')
prophet_model.add_regressor('부동산소비심리')
prophet_model.add_regressor('층')

"""

prophet_model.fit(dff)

In [ ]:
# 미래 예측 - 2018.04 까지 훈련시킨 데이터에 predict 시켜주면서 2018.12까지 예측 해줌

#forecast = prophet_model.predict(dff)
#plot = prophet_model.plot(forecast)

In [ ]:
# 그래프화

future = prophet_model.make_future_dataframe(periods=12, freq='30D')
forecast = prophet_model.predict(future)
forecast
plot = prophet_model.plot(forecast)
#forecast.head()

In [ ]:
# 예측값과 실제값 비교
predicted = forecast['yhat']
actual = alldata['y']
x_plt = forecast['ds'] 
x_labels = [date.strftime('%Y-%m') for date in x_plt] # 연도와 월만 표시되도록 설정

plt.figure(figsize=(30, 10)) # 그래프 크기 조정

plt.plot(predicted, label='Predicted') # 예측값
plt.plot(actual, label='Actual') # 실제값

plt.xticks(range(len(x_labels)), x_labels)
plt.scatter(actual.index, actual, color='red', label='Data Points', s=5)
plt.scatter(predicted.index, predicted, color='blue', label='Data Points', s=5)
plt.xlabel('Date')
plt.ylabel('deposit')
plt.legend()
plt.show()

In [ ]:
# MSE
mse = mean_squared_error(actual, predicted)
print('MSE: ', mse)

# RMSE를 계산(낮을 수록 정확함)
rmse = mean_squared_error(actual, predicted)
print('rmse :',rmse)

# R^2을 계산(R^2가 1에 가까울수록 모델이 데이터를 잘 설명)
r2 = r2_score(actual, predicted)
print('r2 :',r2)

# MAE를 계산(MAE가 낮을수록 예측값과 실제값이 가까워짐)
mae = mean_absolute_error(actual, predicted)
print('mae :',mae)


In [ ]:
#future = prophet_model.make_future_dataframe(periods=12, freq='30D')
#forecast = prophet_model.predict(future)

In [ ]:
#predicted = forecast['yhat']
#actual = dff['y']

#plt.plot(predicted, label='Predicted')
#plt.plot(actual, label='Actual')
#plt.scatter(actual.index, actual, color='red', label='Data Points', s=5)
#plt.scatter(predicted.index, predicted, color='blue', label='Data Points', s=5)
#plt.xlabel('Date')
#plt.ylabel('deposit')
#plt.legend()
#plt.show()